In [22]:
# https://www.analyticsvidhya.com/blog/2019/09/step-by-step-deep-learning-tutorial-video-classification-python/
# Continued from 1.
import keras
from keras.models import Sequential
from keras.applications.vgg16 import VGG16
from keras.layers import Dense, InputLayer, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D, GlobalMaxPooling2D
from keras.preprocessing import image
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.model_selection import train_test_split

In [23]:
train = pd.read_csv('train_new.csv')
train

,image,class
0,ApplyEyeMakeup_g01_c01.avi_frame0,ApplyEyeMakeup
1,ApplyEyeMakeup_g01_c01.avi_frame1,ApplyEyeMakeup
2,ApplyEyeMakeup_g01_c01.avi_frame2,ApplyEyeMakeup
3,ApplyEyeMakeup_g01_c01.avi_frame3,ApplyEyeMakeup
4,ApplyEyeMakeup_g01_c01.avi_frame4,ApplyEyeMakeup
...,...,...
28892,YoYo_g07_c04.avi_frame3,YoYo
28893,YoYo_g07_c04.avi_frame4,YoYo
28894,YoYo_g07_c04.avi_frame5,YoYo
28895,YoYo_g07_c04.avi_frame6,YoYo


In [24]:
# creating an empty list
train_image = []

# for loop to read and store frames
for i in tqdm(range(train.shape[0])):
    # loading the image and keeping the target size as (224,224,3)
    impath = 'scratch/train_01/v_'+train['image'][i]+".jpg"
    if i % 1000 == 0:
        print(impath)
    img = image.load_img(impath, target_size=(224,224,3))
    # converting it to array
    img = image.img_to_array(img)
    # normalizing the pixel value
    img = img/255
    # appending the image to the train_image list
    train_image.append(img)


  0%|          | 15/28897 [00:00<03:17, 146.35it/s]

scratch/train_01/v_ApplyEyeMakeup_g01_c01.avi_frame0.jpg



  4%|▎         | 1068/28897 [00:04<01:28, 312.91it/s]

scratch/train_01/v_BabyCrawling_g05_c03.avi_frame10.jpg



  7%|▋         | 2040/28897 [00:07<01:12, 372.02it/s]

...


 94%|█████████▎| 27035/28897 [01:14<00:05, 361.27it/s]

scratch/train_01/v_TrampolineJumping_g01_c02.avi_frame5.jpg



 97%|█████████▋| 28049/28897 [01:16<00:02, 396.19it/s]

scratch/train_01/v_WalkingWithDog_g03_c03.avi_frame2.jpg



100%|██████████| 28897/28897 [01:18<00:00, 366.60it/s]


In [32]:
# converting the list to numpy array
X = np.array(train_image)

# separating the target
y = train['class']

# creating the training and validation set
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2, stratify = y)
y_train = pd.get_dummies(y_train)
y_test = pd.get_dummies(y_test)

In [33]:
base_model = VGG16(weights='imagenet', include_top=False)

In [34]:
X_train = base_model.predict(X_train)

In [36]:
X_test = base_model.predict(X_test)

In [38]:
X_test.shape

(5780, 7, 7, 512)

In [39]:
X_train.shape

(23117, 7, 7, 512)

In [40]:
X_train = X_train.reshape(23117, 7*7*512)
X_test = X_test.reshape(5780, 7*7*512)

In [44]:
max = X_train.max()
X_train = X_train/max
X_test = X_test/max

In [46]:
model = Sequential()
model.add(Dense(1024, activation='relu', input_shape=(25088,)))
model.add(Dropout(0.5))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(101, activation='softmax'))
# There are 101 in the final layer because there are 101 categories which we would like to put the movies into

In [47]:
from keras.callbacks import ModelCheckpoint
mcp_save = ModelCheckpoint('weight.hdf5', save_best_only=True, monitor='val_loss', mode='min')
model.compile(loss='categorical_crossentropy',optimizer='Adam',metrics=['accuracy'])
# Compile model ready for training

In [48]:
model.fit(X_train, y_train, epochs=200, validation_data=(X_test, y_test), callbacks=[mcp_save], batch_size=128)
# It is okay to stop this training if the accuracy is not getting better.

Epoch 1/200
181/181 [==============================] - 17s 96ms/step - loss: 4.4437 - accuracy: 0.0335 - val_loss: 3.9505 - val_accuracy: 0.1318
Epoch 2/200
181/181 [==============================] - 17s 95ms/step - loss: 3.5634 - accuracy: 0.1790 - val_loss: 2.7417 - val_accuracy: 0.3542
Epoch 3/200
181/181 [==============================] - 17s 95ms/step - loss: 2.7934 - accuracy: 0.3105 - val_loss: 2.0145 - val_accuracy: 0.5176
Epoch 4/200
181/181 [==============================] - 17s 94ms/step - loss: 2.3207 - accuracy: 0.3956 - val_loss: 1.5996 - val_accuracy: 0.6009
Epoch 5/200
181/181 [==============================] - 17s 95ms/step - loss: 2.0132 - accuracy: 0.4555 - val_loss: 1.2716 - val_accuracy: 0.6822
Epoch 6/200
181/181 [==============================] - 17s 95ms/step - loss: 1.7790 - accuracy: 0.5041 - val_loss: 1.0771 - val_accuracy: 0.7419
Epoch 7/200
181/181 [==============================] - 17s 94ms/step - loss: 1.5819 - accuracy: 0.5502 - val_loss: 0.9619 - val_ac

KeyboardInterrupt: 